# OpenAI Agent Example (Tool Calling)

Minimal agent that lets the model call simple tools (current time, calculator, and a stub search), then returns a final answer.

Prereqs:
- Install: `pip install openai`
- Set `OPENAI_API_KEY` in your environment

Optional: set `OPENAI_MODEL` (defaults to `gpt-4o-mini`).

In [1]:
# If needed, uncomment to install
# %pip install -q openai

In [2]:
from __future__ import annotations

import json
import os
from datetime import datetime, timezone
from typing import Any, Dict, List

try:
    from openai import OpenAI
except Exception as e:
    raise SystemExit("The 'openai' package is required. Install with: pip install openai")


## Tools

In [4]:
def tool_get_current_time(_: Dict[str, Any]) -> str:
    """Return current UTC time as ISO 8601 string."""
    return datetime.now(timezone.utc).isoformat()


def _safe_eval_arith(expr: str) -> float:
    """Safely evaluate a simple arithmetic expression.

    Supports numbers, +, -, *, /, **, parentheses. No names or calls.
    """
    import ast
    import operator as op

    allowed_ops = {
        ast.Add: op.add,
        ast.Sub: op.sub,
        ast.Mult: op.mul,
        ast.Div: op.truediv,
        ast.Pow: op.pow,
        ast.USub: op.neg,
        ast.UAdd: lambda x: x,
    }

    def _eval(node):
        if isinstance(node, ast.Num):  # type: ignore[attr-defined]
            return node.n
        if isinstance(node, ast.Constant) and isinstance(node.value, (int, float)):
            return node.value
        if isinstance(node, ast.BinOp) and type(node.op) in allowed_ops:
            return allowed_ops[type(node.op)](_eval(node.left), _eval(node.right))
        if isinstance(node, ast.UnaryOp) and type(node.op) in allowed_ops:
            return allowed_ops[type(node.op)](_eval(node.operand))
        if isinstance(node, ast.Expression):
            return _eval(node.body)
        raise ValueError("Disallowed expression in calculator")

    tree = ast.parse(expr, mode="eval")
    return float(_eval(tree))


def tool_calculator(args: Dict[str, Any]) -> str:
    """Evaluate a basic arithmetic expression and return the numeric result.

    Args schema: {"expression": "3*(7+5)"}
    """
    expr = args.get("expression", "")
    if not isinstance(expr, str) or not expr:
        raise ValueError("Missing 'expression' string for calculator")
    value = _safe_eval_arith(expr)
    return str(value)


def tool_search(args: Dict[str, Any]) -> str:
    """A stub search tool that returns a canned result list.

    Replace with a real search API in your environment if desired.
    Args schema: {"query": "..."}
    """
    query = args.get("query", "")
    results = [
        {"title": "Example Result A", "url": "https://example.com/a"},
        {"title": "Example Result B", "url": "https://example.com/b"},
    ]
    return json.dumps({"query": query, "results": results})


TOOL_REGISTRY = {
    "get_current_time": tool_get_current_time,
    "calculator": tool_calculator,
    "search": tool_search,
}


In [5]:
TOOL_SPECS = [
    {
        'type': 'function',
        'function': {
            'name': 'get_current_time',
            'description': 'Get the current UTC time as ISO 8601 string.',
            'parameters': {'type': 'object', 'properties': {}, 'additionalProperties': False},
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'calculator',
            'description': 'Evaluate a basic arithmetic expression.',
            'parameters': {
                'type': 'object',
                'properties': {'expression': {'type': 'string'}},
                'required': ['expression'],
                'additionalProperties': False,
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'search',
            'description': 'Search the web and return JSON results (stub).',
            'parameters': {
                'type': 'object',
                'properties': {'query': {'type': 'string'}},
                'required': ['query'],
                'additionalProperties': False,
            },
        },
    },
]

SYSTEM_PROMPT = (
    'You are a helpful assistant. '
    'Use tools when they help produce accurate answers. '
    'If you used tools, cite their results succinctly.'
)


## Agent runner

In [6]:
def run_agent(user_input: str, model: str | None = None) -> str:
    """Run a simple tool-using agent for a single user input.

    Returns the assistant's final message content as text.
    """
    if not os.environ.get('OPENAI_API_KEY'):
        raise SystemExit('OPENAI_API_KEY not set in environment')

    client = OpenAI()
    chosen_model = model or os.environ.get('OPENAI_MODEL', 'gpt-4o-mini')

    messages: List[Dict[str, Any]] = [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': user_input},
    ]

    resp = client.chat.completions.create(
        model=chosen_model,
        messages=messages,
        tools=TOOL_SPECS,
        tool_choice='auto',
        temperature=0.2,
    )

    msg = resp.choices[0].message
    # Keep assistant message (including tool_calls, if any)
    try:
        messages.append(msg.dict(exclude_none=True))
    except Exception:
        messages.append({'role': 'assistant', 'content': msg.content, 'tool_calls': msg.tool_calls})

    if msg.tool_calls:
        for call in msg.tool_calls:
            name = call.function.name
            args_json = call.function.arguments or '{}'
            try:
                args = json.loads(args_json)
            except json.JSONDecodeError:
                args = {'_raw': args_json}

            if name not in TOOL_REGISTRY:
                tool_output = f"Tool '{name}' not implemented."
            else:
                try:
                    tool_output = TOOL_REGISTRY[name](args)
                except Exception as e:
                    tool_output = f"Error in tool '{name}': {e}"

            messages.append({
                'role': 'tool',
                'tool_call_id': call.id,
                'name': name,
                'content': tool_output,
            })

        resp = client.chat.completions.create(
            model=chosen_model,
            messages=messages,
            temperature=0.2,
        )

    final_msg = resp.choices[0].message
    return final_msg.content or ''


## Quick test

In [7]:
print(run_agent("What is 3*(7+5), and what time is it?"))

/tmp/ipykernel_145628/240858138.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  messages.append(msg.dict(exclude_none=True))
/tmp/ipykernel_145628/2728057805.py:25: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # type: ignore[attr-defined]
/tmp/ipykernel_145628/2728057805.py:26: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n


The result of \( 3 \times (7 + 5) \) is 36. 

As for the current time, it is 00:36 UTC on August 31, 2025.


## Your own question

In [8]:
query = "Summarize what tools you used and the result."
print(run_agent(query))

I used the following tools:

1. **functions.get_current_time**: This tool retrieves the current UTC time in ISO 8601 format.
2. **functions.calculator**: This tool evaluates basic arithmetic expressions.
3. **functions.search**: This tool searches the web for information based on a query.

However, I did not execute any specific queries or calculations in this instance, so there are no results to summarize. If you have a specific request or question, please let me know!


/tmp/ipykernel_145628/240858138.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  messages.append(msg.dict(exclude_none=True))
